<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

--2023-03-30 22:21:17--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2023-03-30 22:21:17--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2023-03-30 22:21:17--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['dj-khaled.txt',
 'bjork.txt',
 'alicia-keys.txt',
 'bob-dylan.txt',
 'beatles.txt',
 'disney.txt',
 'eminem.txt',
 'r-kelly.txt',
 'amy-winehouse.txt',
 'johnny-cash.txt',
 'janisjoplin.txt',
 'jimi-hendrix.txt',
 'joni-mitchell.txt',
 'leonard-cohen.txt',
 'missy-elliott.txt',
 'lady-gaga.txt',
 'nicki-minaj.txt',
 'radiohead.txt',
 'bruno-mars.txt',
 'Lil_Wayne.txt',
 'adele.txt',
 'dickinson.txt',
 'britney-spears.txt',
 'bieber.txt',
 'kanye.txt',
 'paul-simon.txt',
 'lorde.txt',
 'ludacris.txt',
 'nickelback.txt',
 'michael-jackson.txt',
 'rihanna.txt',
 'patti-smith.txt',
 'notorious_big.txt',
 'cake.txt',
 'dolly-parton.txt',
 'blink-182.txt',
 'nirvana.txt',
 'notorious-big.txt',
 'Kanye_West.txt',
 'lin-manuel-miranda.txt',
 'bruce-springsteen.txt',
 'nursery_rhymes.txt',
 'drake.txt',
 'dr-seuss.txt',
 'al-green.txt',
 'bob-marley.txt',
 'prince.txt',
 'lil-wayne.txt',
 'kanye-west.txt']

In [4]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/eminem.txt', sep='/n', header=None)
df.head()

<ipython-input-4-250d9583aaef>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/eminem.txt', sep='/n', header=None)


,0
0,"Look, I was gonna go easy on you and not to hu..."
1,But I'm only going to get this one chance
2,"Something's wrong, I can feel it (Six minutes,..."
3,"Just a feeling I've got, like something's abou..."
4,"If that means, what I think it means, we're in..."


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 6812


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['look',
  'i',
  'was',
  'gonna',
  'go',
  'easy',
  'on',
  'you',
  'and',
  'not',
  'to',
  'hurt',
  'your',
  'feelings'],
 ['but', "i'm", 'only', 'going', 'to', 'get', 'this', 'one', 'chance']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 6812


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1347


### 3 - Entrenar el modelo generador

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 506556.96875
Loss after epoch 1: 369829.53125
Loss after epoch 2: 344129.125
Loss after epoch 3: 314718.375
Loss after epoch 4: 310399.0
Loss after epoch 5: 295839.5
Loss after epoch 6: 262763.5
Loss after epoch 7: 258724.5
Loss after epoch 8: 254935.0
Loss after epoch 9: 252422.0
Loss after epoch 10: 249889.75
Loss after epoch 11: 246938.0
Loss after epoch 12: 243465.0
Loss after epoch 13: 241613.5
Loss after epoch 14: 229063.75
Loss after epoch 15: 226176.0
Loss after epoch 16: 225148.5
Loss after epoch 17: 223591.5
Loss after epoch 18: 222550.5
Loss after epoch 19: 222643.0


(785273, 1296760)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["mom"], topn=10)

[('loved', 0.8212378025054932),
 ('valium', 0.7913857698440552),
 ('paint', 0.7211800217628479),
 ('dad', 0.7050291895866394),
 ('lots', 0.7019648551940918),
 ('torn', 0.6946876645088196),
 ('self', 0.6901046633720398),
 ("cleanin'", 0.6879480481147766),
 ('skin', 0.6864941716194153),
 ('cleaning', 0.6819820404052734)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["gay"], topn=10)

[('one', -0.2343553900718689),
 ('through', -0.26171961426734924),
 ('from', -0.2733110785484314),
 ('to', -0.29173439741134644),
 ('even', -0.29215461015701294),
 ('as', -0.29296576976776123),
 ('ever', -0.29580244421958923),
 ('been', -0.2983234226703644),
 ('of', -0.3001636564731598),
 ("'til", -0.30290651321411133)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["god"], topn=10)

[('food', 0.7980446815490723),
 ('stamps', 0.7704891562461853),
 ('damn', 0.7608792781829834),
 ('ooh', 0.7356470823287964),
 ('swear', 0.7144322395324707),
 ('thank', 0.7053040862083435),
 ('yea', 0.6808665990829468),
 ('ah', 0.6683138608932495),
 ('sent', 0.6661420464515686),
 ('acid', 0.656876802444458)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('fun', 0.8787831664085388),
 ('safe', 0.8613986372947693),
 ('pedal', 0.8587172627449036),
 ('dawg', 0.8575671911239624),
 ("leavin'", 0.8550981879234314)]

In [18]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["rappers"])

[('never', -0.19924946129322052),
 ('than', -0.20937089622020721),
 ('better', -0.23282310366630554),
 ("ain't", -0.23505836725234985),
 ('more', -0.25058576464653015),
 ('ass', -0.2550211548805237),
 ('there', -0.25547510385513306),
 ('seen', -0.25830182433128357),
 ('bring', -0.26555293798446655),
 ('back', -0.2667425870895386)]

### 5 - Visualizar agrupación de vectores

In [19]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [23]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).

Las palabras relacionadas más cercanas a "mom" son "dad" y "loved". Esto sugiere que hay una relación semántica entre "mom" y "dad" como miembros de una familia, y que "mom" se asocia con sentimientos de afecto. Otras palabras relacionadas como "paint", "cleaning" y "skin" sugieren que "mom" también puede estar asociada con actividades y roles domésticos, como el cuidado del hogar.

Se puede concluir que las palabras menos relacionadas con la palabra "gay" son principalmente preposiciones y adverbios que no tienen una relación semántica o no tienen mucho que ver con la palabra "gay".

- Graficarlos.

- Obtener conclusiones.

Se puede observar que las palabras que se relacionan más entre sí se agrupan, o al menos se ubican cerca. Aunque en mi parecer la distribución de las palabras del vocabulario esta bastante dispersa.